<a href="https://colab.research.google.com/github/iniola145/16-prettytable-module-py/blob/main/Cassava_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install Kaggle API
!pip install -q kaggle

# Upload Kaggle API key (must be done manually)
from google.colab import files
files.upload()  # Upload kaggle.json

# Move kaggle.json to the correct directory and set permissions
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download Cassava dataset
!kaggle datasets download -d mexwell/crop-diseases-classification
!unzip -q crop-diseases-classification.zip -d Data/


Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/mexwell/crop-diseases-classification
License(s): other
100% 1.99G/2.00G [00:13<00:00, 222MB/s]
100% 2.00G/2.00G [00:13<00:00, 163MB/s]
replace Data/Data/label_num_to_disease_map.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace Data/Data/train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace Data/Data/train_images/157078263.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace Data/Data/train_images/1574893536.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace Data/Data/train_images/1575013487.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace Data/Data/train_images/1576606254.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace Data/Data/train_images/1579761476.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
Y
y
A y


In [4]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split

# Set dataset paths
csv_path = "/content/Data/train.csv"
img_dir = "/content/Data/train_images"

# Load CSV
df = pd.read_csv(csv_path)

# Remove missing images
df["filepath"] = df["image_id"].apply(lambda x: os.path.join(img_dir, x))
df = df[df["filepath"].apply(os.path.exists)]  # Keep only existing files

# Split into train & validation sets (80/20)
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df["label"], random_state=42)

print(f"Total images after filtering: {len(df)}")
print(f"Train size: {len(train_df)}, Validation size: {len(val_df)}")


Total images after filtering: 17938
Train size: 14350, Validation size: 3588


In [3]:
import os

# Check if the Data folder exists
print("Data directory exists:", os.path.exists("/content/Data/"))

# Check if train.csv exists
print("train.csv exists:", os.path.exists("/content/Data/train.csv"))

# List contents of Data folder
!ls -l /content/Data/


Data directory exists: True
train.csv exists: False
total 4
drwxr-xr-x 3 root root 4096 Mar 17 16:26 Data


In [16]:
import os
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image

# Set dataset path
dataset_path = "/content/Data"
csv_path = os.path.join(dataset_path, "train.csv")
image_folder = os.path.join(dataset_path, "train_images")

# Load CSV file
df = pd.read_csv(csv_path)

# Remove missing image paths
df['image_path'] = df['image_id'].apply(lambda x: os.path.join(image_folder, x))
df = df[df['image_path'].apply(os.path.exists)]  # Remove rows where image file doesn't exist

print(f"Total images after filtering: {len(df)}")

# Split dataset into train (80%) and validation (20%)
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

# Define transforms
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Custom dataset class
class CassavaDataset(Dataset):
    def __init__(self, dataframe, transform):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx]['image_path']
        label = self.dataframe.iloc[idx]['label']

        image = Image.open(img_path).convert("RGB")
        image = self.transform(image)

        return image, label

# Create PyTorch Datasets
train_dataset = CassavaDataset(train_df, train_transforms)
val_dataset = CassavaDataset(val_df, val_transforms)

# Adjust batch size based on GPU memory (reduce if Colab crashes)
batch_size = 16  # Change to 8 if OOM occurs

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

print("Data preprocessing complete! 🚀")


Total images after filtering: 17938
Data preprocessing complete! 🚀


In [21]:
import torch.nn as nn
import torchvision.models as models

# Feature extractor class
class FeatureExtractor(nn.Module):
    def __init__(self, backbone):
        super(FeatureExtractor, self).__init__()
        self.backbone = backbone
        self.global_pool = nn.AdaptiveAvgPool2d((1, 1))  # Global average pooling

    def forward(self, x):
        x = self.backbone(x)
        x = self.global_pool(x)  # Apply global average pooling
        x = torch.flatten(x, 1)  # Flatten into a single vector
        return x

# Load EfficientNetV2-S (pretrained)
efficientnet = models.efficientnet_v2_s(weights="IMAGENET1K_V1")
efficientnet_features = efficientnet.features  # Extract feature layers

# Load CropNet (Replace with real CropNet model)
cropnet = models.mobilenet_v2(weights="IMAGENET1K_V1")  # Placeholder for CropNet
cropnet_features = cropnet.features  # Extract feature layers

# Define feature extraction modules
efficientnet_extractor = FeatureExtractor(efficientnet_features)
cropnet_extractor = FeatureExtractor(cropnet_features)

# Define Cassava Classifier
class CassavaClassifier(nn.Module):
    def __init__(self):
        super(CassavaClassifier, self).__init__()
        self.efficientnet = efficientnet_extractor
        self.cropnet = cropnet_extractor

        # Check feature output size
        test_input = torch.randn(1, 3, 224, 224)  # Fake input for testing
        with torch.no_grad():
            feat1_size = self.efficientnet(test_input).shape[1]
            feat2_size = self.cropnet(test_input).shape[1]

        # Fully connected classifier
        self.fc = nn.Sequential(
            nn.Linear(feat1_size + feat2_size, 512),  # Match correct feature sizes
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 5)  # 5 classes for Cassava disease
        )

    def forward(self, x):
        feat1 = self.efficientnet(x)
        feat2 = self.cropnet(x)
        fused_features = torch.cat((feat1, feat2), dim=1)  # Concatenate features
        return self.fc(fused_features)

# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CassavaClassifier().to(device)

print("✅ Model fixed and initialized successfully!")


✅ Model fixed and initialized successfully!


In [22]:
import torch.optim as optim

# Define loss function & optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)

# Mixed Precision Training (AMP)
scaler = torch.cuda.amp.GradScaler()

print("Optimizer and AMP setup complete! 🚀")


Optimizer and AMP setup complete! 🚀


<ipython-input-22-d94d5177d012>:8: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


In [23]:
def train_one_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0

    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        # Mixed precision training
        with torch.amp.autocast(device_type="cuda"):
            outputs = model(images)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()

    return running_loss / len(dataloader)

def validate(model, dataloader, criterion, device):
    model.eval()
    correct = 0
    total = 0
    running_loss = 0.0

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)

            with torch.amp.autocast(device_type="cuda"):
                outputs = model(images)
                loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    accuracy = 100 * correct / total
    return running_loss / len(dataloader), accuracy

print("Training & validation functions ready! 🚀")


Training & validation functions ready! 🚀


In [24]:
num_epochs = 10
best_val_acc = 0.0

for epoch in range(num_epochs):
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc = validate(model, val_loader, criterion, device)

    print(f"Epoch {epoch+1}: Train Loss: {train_loss:.4f}, Val Acc: {val_acc:.2f}%")

    # Save the best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_cassava_model.pth")
        print("✅ Best model saved!")

print("Training complete! 🚀")


Epoch 1: Train Loss: 0.6359, Val Acc: 83.72%
✅ Best model saved!
Epoch 2: Train Loss: 0.4625, Val Acc: 85.12%
✅ Best model saved!
Epoch 3: Train Loss: 0.3958, Val Acc: 84.56%
Epoch 4: Train Loss: 0.3364, Val Acc: 85.17%
✅ Best model saved!
Epoch 5: Train Loss: 0.2774, Val Acc: 84.84%
Epoch 6: Train Loss: 0.2275, Val Acc: 84.73%
Epoch 7: Train Loss: 0.1830, Val Acc: 83.42%
Epoch 8: Train Loss: 0.1584, Val Acc: 84.31%
Epoch 9: Train Loss: 0.1276, Val Acc: 84.06%
Epoch 10: Train Loss: 0.1118, Val Acc: 84.95%
Training complete! 🚀
